In [4]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

In [5]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


In [ ]:
#output for the skeleton edges to be stored by datajoint
""" OLD WAY THAT DATAJOINT WAS GETTING MAD AT 
def read_skeleton(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            bones.append(np.array(line.split()[1:], float).reshape(-1, 3))
    return np.array(bones)
"""

""" NEW FLAT LIST WAY"""
#practice reading in dummy skeleton file
def read_skeleton_flat(file_path):
    with open(file_path) as f:
        bones = list()
        for line in f.readlines():
            for r in (np.array(line.split()[1:], float).reshape(-1, 3)):
                bones.append(r)
            bones.append([np.nan,np.nan,np.nan])
    return np.array(bones).astype(float)


In [ ]:
#make sure there is a temp file in the directory, if not then make one
#if temp folder doesn't exist then create it
if (os.path.isdir(os.getcwd() + "/temp_meshlab")) == False:
    os.mkdir("temp_meshlab")
    
    

In [ ]:
def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    subprocess_result = subprocess.run('xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + 
                   script_command,shell=True)
    
    return subprocess_result

In [ ]:
def meshlab_shrinkwrap(key):
    
    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)
    
    
    input_mesh = path_and_filename + ".off"
    midoutput_mesh = path_and_filename + "_mid.off"
    output_mesh = path_and_filename+"_mls.off"
    
    
    meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_script.mlx"
    meshlab_script_rem_dupl = str(pathlib.Path.cwd()) + "/" + "remeshing_script-Remove.mlx"
    #send to meshlabserver
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      midoutput_mesh)
    print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass Poisson')
    print(type)
     
    
    #do another call to remove the final duplicate vertices
    print("starting meshlabserver Poisson surface reconstruction")
    subprocess_result_2 = run_meshlab_script(meshlab_script_rem_dupl,
                      midoutput_mesh,
                      output_mesh)
    print("Cleaning subprocess_result= "+ str(subprocess_result_2))
    
    if str(subprocess_result_2)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(key["segment_id"]) + 
                         ' did not get pass cleaning')
    
    
    return output_mesh

In [ ]:
#do a testrun for this neurite
key = dict(segment_id=648518346341351525,segmentation=2)



In [ ]:
#don't need the vertices for this one because not doing any filtering
def make_skeleton(key):
    global_time = time.time()
    #get the mesh with the error segments filtered away
    start_time = time.time()

    file_loc = pathlib.Path.cwd() / "temp_meshlab"
    filename = "neuron_" + str(key["segment_id"])
    path_and_filename = str(file_loc / filename)

    #get the mesh of the neurite
    new_key = dict(segmentation=key["segmentation"],
                   segment_id=key["segment_id"])
    mesh = (ta3p100.Mesh & new_key).fetch1()
    print(f"Step 1: Retrieving Mesh: {time.time() - start_time}")

    #Do voxelization
    start_time = time.time()
    mesh = trimesh_io.Mesh(vertices=mesh["vertices"], faces=mesh["triangles"])
    voxels = mesh.voxelized(500)
    voxel_mesh = voxels.as_boxes()
    print(f"Step 2: Voxelization: {time.time() - start_time}")

    #Exporting the Voxelization as an off file for meshlabserver
    start_time = time.time()
    #try the inline printing method:
    with open(os.devnull, 'w') as devnull:
        with contextlib.redirect_stdout(devnull):
            voxel_mesh.export(str(path_and_filename) + ".off")
    print(f"Step 3: Exporting Voxel Off function: {time.time() - start_time}")

    #Run the meshlabserver scripts
    start_time = time.time()
    output_mesh = meshlab_shrinkwrap(key)
    print(f"Step 4: Meshlab shrinkwrapping: {time.time() - start_time}")

    #Create the skeleton and retrieve it from the generated file
    #skeletonize the mesh
    start_time = time.time()
    print("starting creating skeleton")
    return_value = cm.calcification(output_mesh[:-4])
    print("calcif_return_value = " + str(return_value))

    if return_value > 0:
        raise Exception('skeletonization for neuron ' + str(new_key["segment_id"]) + 
                        ' did not finish... exited with error code: ' + str(return_value))
    print(f"Step 4: Generating Skeleton: {time.time() - start_time}")

    #read in the skeleton files into an array
    start_time = time.time()
    bone_array = read_skeleton_flat(output_mesh[:-4]+"_skeleton.cgal")
    #print(bone_array)
    if len(bone_array) <= 0:
        raise Exception('No skeleton generated for ' + str(new_key["segment_id"]))
    print(f"Step 5: Reading in Skeleton: {time.time() - start_time}")


    start_time = time.time()
    new_key["n_branches"] = len(bone_array)
    new_key["branches"] = bone_array
    #new_key["branches"] = []


    key["time_updated"]=str(datetime.datetime.now())
    print(key)
    #if all goes well then write to database
#     self.insert1(key)
#     print(f"Step 6: Inserting dictionary: {time.time() - start_time}")
#     print(f"Total time: {time.time() - global_time}")
#     print("\n\n")
    
    #erase the files used
    os.system("rm "+str(path_and_filename)+"*")

In [ ]:
make_skeleton(key)

In [ ]:
#erase the files used
#os.system("rm "+str(path_and_filename)+"*")
str(path_and_filename)

In [ ]:
file_loc = pathlib.Path.cwd() / "temp_meshlab"
filename = "neuron_" + str(key["segment_id"])
path_and_filename = str(file_loc / filename)
str(path_and_filename)

In [ ]:
file_loc = pathlib.Path.cwd() / "temp_meshlab"
filename = "neuron_" + str(key["segment_id"])
path_and_filename = str(file_loc / filename)
str(path_and_filename)
os.system("rm "+str(path_and_filename)+"*")

In [9]:
ID = 543
primary_key = dict(segmentation=2,decimation_ratio=0.35,segment_id=ID)
#neuron_data = ((ta3p100.CleansedMesh  & primary_key).fetch(as_dict=True))[0]

In [11]:
len(ta3p100.CleansedMesh() & primary_key)

0

In [13]:
ta3p100.NeuriteSkeletons()

segmentation segmentation id,segment_id segment id unique within each Segmentation,time_updated the time at which the skeleton was generated,n_branches number of edges stored,branches array storing vertices of edges and each seperated by Nan
2,648518346341351518,2019-03-19 05:27:05,362,=BLOB=
2,648518346341351523,2019-03-19 05:27:10,271,=BLOB=
2,648518346341351525,2019-03-19 05:27:17,322,=BLOB=
2,648518346341351531,2019-03-19 05:27:22,317,=BLOB=
2,648518346341351631,2019-03-19 05:27:29,872,=BLOB=
2,648518346341351638,2019-03-19 05:27:34,299,=BLOB=
2,648518346341351695,2019-03-19 05:27:39,450,=BLOB=
2,648518346341351710,2019-03-19 05:27:45,288,=BLOB=
2,648518346341351713,2019-03-19 05:27:51,253,=BLOB=
2,648518346341351718,2019-03-19 05:27:58,302,=BLOB=


In [14]:
neurite_data = (ta3p100.NeuriteSkeletons &"segment_id=648518346341351771").fetch1()

In [17]:
neurite_data["branches"]

array([[252264. , 257211. ,  44540. ],
       [252533. , 257176. ,  44884.9],
       [252707. , 257229. ,  45031. ],
       ...,
       [234405. , 232664. ,  61020.3],
       [234219. , 233113. ,  61190.8],
       [     nan,      nan,      nan]])

In [20]:
a = [ 272991.  , 191725.  ,  45119.7]
b = [np.nan, np.nan, np.nan]

In [26]:
a != b

True

In [22]:
ordered_branches = [[]]

In [23]:
ordered_branches == [[]]

True

In [25]:
ordered_branches[0] == b

False

In [46]:
branches = neurite_data["branches"]

In [51]:
ordered_branches = [[]]
for counter,current_vert in enumerate(branches):
    print(current_vert)

    #if not((current_vert==np.ndarray([np.nan,np.nan,np.nan])).all()):
    if np.array_equal(current_vert,np.ndarray([np.nan,np.nan,np.nan])):
        ordered_branches[branch_counter].append(current_vert)
    else:
        branch_counter += 1
        ordered_branch.append([])

[252264. 257211.  44540.]


TypeError: 'float' object cannot be interpreted as an integer

In [71]:
#cv = np.ndarray([np.nan,np.nan,np.nan])
print(np.nan == cv[0])
np.isnan(cv[0])

False


True

In [79]:
#assemble the reader list
branch_counter = 0
ordered_branches = [[]]
for counter,current_vert in enumerate(branches):
    #print(current_vert)

    if not(np.isnan(current_vert[0])):
        ordered_branches[branch_counter].append(current_vert)
    else:
        branch_counter += 1
        ordered_branches.append([])

ordered_branches = ordered_branches[:-1]

In [80]:
ordered_branches

[[array([252264., 257211.,  44540.]),
  array([252533. , 257176. ,  44884.9]),
  array([252707., 257229.,  45031.]),
  array([252944. , 257302. ,  45229.6]),
  array([253236. , 257404. ,  45499.1]),
  array([253471. , 257522. ,  45697.2]),
  array([253624. , 257592. ,  45854.9]),
  array([253760., 257675.,  45998.]),
  array([253922. , 257771. ,  46162.3]),
  array([254002. , 257826. ,  46251.2]),
  array([254242. , 258001. ,  46506.5]),
  array([254444. , 258164. ,  46715.1]),
  array([254625. , 258319. ,  46889.6]),
  array([254745. , 258419. ,  47003.1]),
  array([254896. , 258547. ,  47131.6]),
  array([255119. , 258720. ,  47319.3]),
  array([255308. , 258862. ,  47437.8]),
  array([255514. , 259021. ,  47564.4]),
  array([255795. , 259189. ,  47717.2]),
  array([255991. , 259317. ,  47822.4]),
  array([256149. , 259414. ,  47908.5]),
  array([256291. , 259493. ,  47977.5]),
  array([256680. , 259734. ,  48193.6]),
  array([256819. , 259818. ,  48266.6]),
  array([257027. , 259930

In [81]:
len(ordered_branches[1])

59